DOWNLOAD YOUTUBE VIDEO

In [5]:
import yt_dlp

# Enter the YouTube video URL
url = "https://www.youtube.com/watch?v=NaDfRqFwKVM&ab_channel=NBCNews"

ydl_opts = {
    "outtmpl": "videos/%(title)s.%(ext)s",  # save as video title
    "format": "best"
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])


[youtube] Extracting URL: https://www.youtube.com/watch?v=NaDfRqFwKVM&ab_channel=NBCNews
[youtube] NaDfRqFwKVM: Downloading webpage
[youtube] NaDfRqFwKVM: Downloading tv simply player API JSON
[youtube] NaDfRqFwKVM: Downloading tv client config
[youtube] NaDfRqFwKVM: Downloading tv player API JSON
[info] NaDfRqFwKVM: Downloading 1 format(s): 18
[download] Sleeping 3.00 seconds as required by the site...
[download] Destination: videos\Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp4
[download] 100% of    4.83MiB in 00:00:00 at 7.06MiB/s   


DOWNLOAD AUDIO FROM YOUTUBE VIDEO

In [6]:
import yt_dlp

url = "https://www.youtube.com/watch?v=NaDfRqFwKVM&ab_channel=NBCNews"

ydl_opts = {
    "format": "bestaudio/best",
    "outtmpl": "Audios/%(title)s.%(ext)s",
    "postprocessors": [
        {
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",   # or "wav"
            "preferredquality": "192", # kbps
        }
    ],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

print("✅ Audio extracted directly as MP3!")


[youtube] Extracting URL: https://www.youtube.com/watch?v=NaDfRqFwKVM&ab_channel=NBCNews
[youtube] NaDfRqFwKVM: Downloading webpage
[youtube] NaDfRqFwKVM: Downloading tv simply player API JSON
[youtube] NaDfRqFwKVM: Downloading tv client config
[youtube] NaDfRqFwKVM: Downloading tv player API JSON
[info] NaDfRqFwKVM: Downloading 1 format(s): 251
[download] Sleeping 3.00 seconds as required by the site...
[download] Destination: Audios\Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.webm
[download] 100% of    1.20MiB in 00:00:00 at 3.22MiB/s   
[ExtractAudio] Destination: Audios\Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp3
Deleting original file Audios\Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.webm (pass -k to keep)
✅ Audio extracted directly as MP3!


EXTRACT AUDIO FROM LOCAL VIDEO

In [1]:
from moviepy import VideoFileClip

# Path to downloaded video
video_path = "videos/Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp4"
output_audio_path = "Audios/sample_audio.mp3"

# Load video and extract audio
video = VideoFileClip(video_path)
video.audio.write_audiofile(output_audio_path)

print("✅ Audio extracted as MP3!")


MoviePy - Writing audio in Audios/sample_audio.mp3


MoviePy - Done.
✅ Audio extracted as MP3!


GENERATE EMBEDDINGS

In [11]:
from torchvision import models, transforms
from PIL import Image
import torch
import cv2

# Load pre-trained model (e.g., ResNet)
model = models.resnet50(pretrained=True)
model.eval()

# Video path
video_path = "videos/Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp4"
cap = cv2.VideoCapture(video_path)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

embeddings = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    img = transform(frame).unsqueeze(0)
    with torch.no_grad():
        emb = model(img)
    embeddings.append(emb)

cap.release()
print(f"Created {len(embeddings)} frame embeddings")


Created 2821 frame embeddings


In [13]:
import numpy as np
np.array(embeddings).shape

(2821, 1, 1000)

In [ ]:
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
import torch

image = Image.open("frame.jpg")
extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = ViTModel.from_pretrained("google/vit-base-patch16-224")

inputs = extractor(images=image, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
embedding = outputs.last_hidden_state.mean(dim=1)  # pooled embedding


LIMB POSITION

In [15]:
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")
results = model("videos/Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp4")

for frame in results:
    keypoints = frame.keypoints  # x, y, confidence



WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\Michael Phelps reflects on depression and mental health 'I saw it as a sign of weakness'.mp4: 384x640 2 persons, 85.8ms
video 1/1 (frame 2/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\Michael Phelps reflects on depression and mental health 'I saw it as a sign of weakness'.mp4: 384x640 2 persons, 20.3ms
video 1/1 (frame 3/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\

In [21]:
from ultralytics import YOLO
import cv2

model = YOLO("yolov8n-pose.pt")

cap = cv2.VideoCapture("videos/Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp4")

while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    # Run pose inference on current frame
    results = model(frame, verbose=False)

    for r in results:
        if r.keypoints is not None:
            kps = r.keypoints.xy.cpu().numpy()   # (n_persons, n_joints, 2)
            conf = r.keypoints.conf.cpu().numpy()
            print("Keypoints shape:", kps.shape)
            print("Keypoints:", kps)
        else:
            print("No person detected in this frame.")

cap.release()


Keypoints shape: (2, 17, 2)
Keypoints: [[[     161.57       55.34]
  [     163.38      49.425]
  [     153.64      47.801]
  [     157.79      55.823]
  [     131.02      53.572]
  [     156.16      98.107]
  [     111.97      104.54]
  [     176.88      156.02]
  [     130.68      177.28]
  [     193.54       191.8]
  [     188.69      197.99]
  [     171.16      196.57]
  [     141.82      208.08]
  [     256.45      219.47]
  [        227      237.25]
  [      226.6      290.31]
  [     220.07      314.71]]

 [[     450.03      83.742]
  [     451.08      78.086]
  [     454.05       77.94]
  [        465      81.291]
  [     477.26      80.387]
  [     487.45      117.08]
  [     492.23      114.72]
  [     480.07      174.33]
  [     473.94      161.39]
  [     445.54      193.92]
  [     449.79      134.97]
  [      477.5      207.77]
  [     482.19      198.83]
  [     403.06      207.19]
  [      406.3      198.97]
  [     362.22      295.72]
  [     362.71      280.76]]]
Keypo

In [22]:
for r in results:
    kps = r.keypoints.xy.cpu().numpy()   # (n_persons, n_joints, 2)
    conf = r.keypoints.conf.cpu().numpy()  # confidence
    print(kps)  # x,y coords of joints


[[[     350.08      83.841]
  [     363.98      70.462]
  [     336.42       69.49]
  [     382.02       78.52]
  [     316.87      76.579]
  [     410.06      160.16]
  [     279.87      159.45]
  [     433.37       258.1]
  [      257.1      263.89]
  [      402.6      318.43]
  [     323.11       323.6]
  [     388.84      341.05]
  [     303.72      341.43]
  [      396.7         360]
  [     299.01      353.43]
  [     395.74         360]
  [      351.6      339.57]]]


In [23]:
import numpy as np

prev_kps = None
for r in results:
    kps = r.keypoints.xy.cpu().numpy()

    if prev_kps is not None:
        movement = np.linalg.norm(kps - prev_kps, axis=2)  # per joint movement
        hand_movement = movement[:, [9, 10]]  # wrists only (left=9, right=10)
        print("Hand Movement:", hand_movement)

    prev_kps = kps


In [26]:
from ultralytics import YOLO
import cv2

# Load YOLO pose model
model = YOLO("yolov8n-pose.pt")

# Open input video
cap = cv2.VideoCapture("videos/Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video writer
out = cv2.VideoWriter("pose_output.mp4",
                      cv2.VideoWriter_fourcc(*"mp4v"),
                      fps, (w, h))

while True:
    ret, frame = cap.read()
    if not ret:
        break  # end of video

    # Run inference
    results = model(frame, verbose=False)

    for r in results:
        if r.keypoints is not None:
            kps = r.keypoints.xy.cpu().numpy()   # (n_persons, n_joints, 2)
            conf = r.keypoints.conf.cpu().numpy()

            # Draw joints (keypoints)
            for person in kps:
                for (x, y) in person:
                    cv2.circle(frame, (int(x), int(y)), 4, (0, 255, 0), -1)

    # Write annotated frame
    out.write(frame)

    # (Optional) Show live
    cv2.imshow("Pose Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Pose video saved as pose_output.mp4")


✅ Pose video saved as pose_output.mp4


EMOTION DETECTION

In [ ]:
import cv2
from deepface import DeepFace
import matplotlib.pyplot as plt
from collections import Counter

# --- Input / Output paths ---
video_path = "video.mp4"
output_video_path = "emotion_output.mp4"

# --- Open video ---
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# --- Video writer ---
out = cv2.VideoWriter(output_video_path,
                      cv2.VideoWriter_fourcc(*"mp4v"),
                      fps, (w, h))

frame_count = 0
emotions_over_time = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    try:
        # Analyze emotion for this frame
        result = DeepFace.analyze(frame, actions=["emotion"], enforce_detection=False)
        dominant_emotion = result[0]["dominant_emotion"]

        # Store for later graph
        emotions_over_time.append(dominant_emotion)

        # Overlay emotion text on frame
        cv2.putText(frame, f"Emotion: {dominant_emotion}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    except Exception as e:
        # No face detected, skip
        emotions_over_time.append("none")

    # Write annotated frame
    out.write(frame)

cap.release()
out.release()

print("✅ Emotion-annotated video saved:", output_video_path)

# --- Overall dominant emotion ---
filtered_emotions = [e for e in emotions_over_time if e != "none"]
overall_emotion = Counter(filtered_emotions).most_common(1)[0][0]
print("🎭 Overall Emotion Detected:", overall_emotion)

# --- Plot emotion variation over frames ---
plt.figure(figsize=(12, 5))
plt.plot(emotions_over_time, marker="o")
plt.title("Emotion Variation Over Time")
plt.xlabel("Frame Number")
plt.ylabel("Detected Emotion")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
